## API Design

In [7]:
%%writefile server.py

import Antenna as ant
import os
import json
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename


UPLOAD_FOLDER = './kml'
ALLOWED_EXTENSIONS = set(['kml'])

app = Flask("agra")
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['ENV'] = 'development'
app.config['DEBUG'] = True
app.config['TESTING'] = True

# ----------------------- web services
def compute_from_kml(path,filename):
    csvfile = './csv/'+ filename + ".csv"
    ant.transform_kml_to_csv(path, csvfile, filename)
    soldf = ant.request_best_route(csvfile, filename)    
    # todo :erase kml and csv path after use
    return(soldf.to_json())

@app.route('/svc/kml', methods = ['GET', 'POST'])
def svc_process_kml():
    if request.method == 'POST':
        f = request.files['file']
        filename = secure_filename(f.filename)
        path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(path)
        return(compute_from_kml(path, filename))
    
    
# ----------------------- gui services
def render_from_kml(path,filename):
    csvfile = './csv/'+ filename + ".csv"
    ant.transform_kml_to_csv(path, csvfile, filename)
    df = ant.request_best_route(csvfile, filename)
    js = json.loads(df.to_json())
    return( 
        render_template(
            'result.html', 
            stamps = js['stamp'],
            locs = js['loc'],
            lats = js['lat'],
            lons = js['lon'],
            rowkeys = [str(i) for i in range(len(js['stamp']))]
        )
    )

@app.route('/gui/kml/process', methods = ['GET', 'POST'])
def gui_process_kml():
    if request.method == 'POST':
        f = request.files['file']
        filename = secure_filename(f.filename)
        path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(path)
        return(render_from_kml(path, filename))

@app.route('/gui/kml')
def query_file():
    return render_template('upload.html')

@app.route('/gui/form')
def query_form():
    return render_template('form.html')

@app.route('/gui/json/coordinates', methods = ['GET','POST'])
def render_from_json_coordinates():
    filename = 'mawar'
    csvfile = './csv/'+ filename + ".csv"
    if request.method == 'POST':
        js = request.get_json()
        ant.transform_list_to_csv(js['coordinates'], csvfile, filename)
        df = ant.request_best_route(csvfile, filename)
        js = json.loads(df.to_json())
        return(
            render_template(
                'result.html', 
                stamps = js['stamp'],
                locs = js['loc'],
                lats = js['lat'],
                lons = js['lon'],
                rowkeys = [str(i) for i in range(len(js['stamp']))]
            )
        )        
app.run()


Overwriting server.py


In [22]:
%%writefile templates/upload.html
<html>
    <body>
    <h1>Submit KML File:</h1>
    <form action = "http://localhost:5000/uploader" method = "POST" 
        enctype = "multipart/form-data">
        <input type = "file" name = "file" />
        <input type = "submit"/>
    </form>
   </body>
</html>

Overwriting templates/upload.html


In [14]:
import Antenna as ant
import os
import json

df = ant.request_best_route('csv/mawar.csv', 'mawar')

Time Table:

 [[ 0 11  9 16 13]
 [ 0  0  8  8  9]
 [ 0  5  0  9  9]
 [ 0 10  9  0  7]
 [ 0  5  8  7  0]]

SOLUTION FOUND ON ROUTE:
---------------------

depo Time(0) -> tgt2 Time(9) -> tgt1 Time(14) -> tgt3 Time(22) -> tgt4 Time(29) -> END
Total time of all routes: 29min
--- 0.0009868144989013672 seconds ---
